In [1]:
from tqdm.notebook import tqdm
import pandas as pd
from datasets import Dataset
from src.dataset import SPFastaDatasetBinary, SPFastaDatasetBinaryWithTokenizedCategory
import torch
from torch import nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AdamW
from torch.utils.data import DataLoader
import numpy as np
import os
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# ds = SPFastaDatasetBinaryWithTokenizedCategory("data/train.fasta")
# dataset_train = Dataset.from_pandas(ds.data).with_format("torch", device=device)
# ds = SPFastaDatasetBinaryWithTokenizedCategory("data/test.fasta")
# dataset_test = Dataset.from_pandas(ds.data).with_format("torch", device=device)
# del ds

In [3]:
def load_data_with_size_uniform(path, size=1000):
    ds = SPFastaDatasetBinaryWithTokenizedCategory(path)
    neg_labels = ds.data.loc[ds.data['labels'] == 0][:size//2]
    pos_labels = ds.data.loc[ds.data['labels'] == 1][:size//2]
    combi = pd.concat((neg_labels, pos_labels))
    return Dataset.from_pandas(combi).with_format("torch", device=device)

dataset_train = load_data_with_size_uniform("data/train.fasta", 100000)
dataset_test  = load_data_with_size_uniform("data/test.fasta", 100000)

100%|██████████| 8811/8811 [00:00<00:00, 734434.56it/s]


In [4]:
from datasets import ClassLabel
dataset_train = dataset_train.cast_column('labels', ClassLabel(num_classes = 2, names=["NO_SP", "SP"]))
dataset_test = dataset_test.cast_column('labels', ClassLabel(num_classes = 2, names=["NO_SP", "SP"]))
dataset_train[0]['labels']

Casting the dataset:   0%|          | 0/20290 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/8811 [00:00<?, ? examples/s]

tensor(0, device='cuda:0')

In [5]:
# Tokenize:
tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert", device="cpu")
dataset_train = dataset_train.map(lambda x: tokenizer(x['text'], return_tensors="pt", padding='max_length', max_length=81, truncation=True), batched=True)
dataset_test = dataset_test.map(lambda x: tokenizer(x['text'], return_tensors="pt", padding='max_length', max_length=81, truncation=True), batched=True)
dataset_train, dataset_test

Map:   0%|          | 0/20290 [00:00<?, ? examples/s]

Map:   0%|          | 0/8811 [00:00<?, ? examples/s]

(Dataset({
     features: ['text', 'labels', 'uniprot_ac', 'kingdom', 'type', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 20290
 }),
 Dataset({
     features: ['text', 'labels', 'uniprot_ac', 'kingdom', 'type', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 8811
 }))

In [6]:
import evaluate
acc = evaluate.load("accuracy")
pre = evaluate.load("precision")
rec = evaluate.load("recall")
f1m = evaluate.load("f1")

metrics = [acc, pre, rec, f1m]

def compute_metrics(eval_pred):
    logits, y = eval_pred
    x = logits.argmax(-1)
    return {k: v for metric in [m.compute(predictions=x, references=y) for m in metrics] for k, v in metric.items()}

# Define model

In [7]:
from transformers import AutoModelForSequenceClassification

class ProtBertSequenceClassification(nn.Module):
    def __init__(self, device: str=device) -> None:
        super(ProtBertSequenceClassification, self).__init__()
        self.device = device

        self.model = AutoModelForSequenceClassification.from_pretrained(
            #"Rostlab/prot_bert",
            "microsoft/MiniLM-L12-H384-uncased",
            num_labels=2,
            label2id = {
                'NO_SP': 0,
                'SP': 1
            },
            id2label= {
                0: 'NO_SP',
                1: 'SP'
            }).to(device)

        self.loss_fn = nn.functional.binary_cross_entropy

    def forward(self, input_ids, attention_mask=None, labels=None, token_type_ids=None):
        outputs = self.model.forward(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask
        )
        return outputs
    
    def loss(self, x, weights=None):
        logits = self.forward(x).logits[:,1]
        pred = logits.softmax(-1)
        label = x['labels']
        return self.loss_fn(pred, label.float(), weight=weights)

model = ProtBertSequenceClassification()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Define FocalLoss, compute class weights and create a custom Trainer

In [8]:
class FocalLoss(nn.Module):
    def __init__(self, weights, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = weights
        self.gamma = gamma

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        loss = (self.alpha[targets] * (1 - pt) ** self.gamma * ce_loss).mean()
        return loss

In [9]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=dataset_train['labels'].cpu().numpy())
class_weights = torch.tensor(class_weights).to(device).float()
class_weights

tensor([0.6493, 2.1747], device='cuda:0')

In [10]:
from transformers import Trainer

class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        
        outputs = model(**inputs)
        logits = outputs.get('logits')
        labels = inputs.get('labels')

        loss_fn = FocalLoss(weights=class_weights)
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [11]:
from transformers import TrainingArguments

# batch_size = 8
epochs = 50
logging_steps=1000
eval_steps=10000

args = TrainingArguments(
    output_dir='./model',
    logging_dir="./logs",
    num_train_epochs=epochs,
    learning_rate=2e-5,
    logging_first_step=True,
    weight_decay=0.0001,
    evaluation_strategy='steps',
    save_strategy='epoch',
    logging_steps=logging_steps,
    eval_steps=eval_steps,
    fp16=True,
    optim='adamw_torch',
    remove_unused_columns=True,
    auto_find_batch_size=True,
)

In [12]:
trainer = WeightedLossTrainer(
    model=model,
    args=args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    tokenizer=tokenizer
)
trainer.train()

  0%|          | 0/126850 [00:00<?, ?it/s]

{'loss': 0.1488, 'learning_rate': 1.9999842333464724e-05, 'epoch': 0.0}
{'loss': 0.1737, 'learning_rate': 1.9842333464722114e-05, 'epoch': 0.39}
{'loss': 0.1713, 'learning_rate': 1.968498226251478e-05, 'epoch': 0.79}
{'loss': 0.1741, 'learning_rate': 1.9527315727236894e-05, 'epoch': 1.18}
{'loss': 0.1357, 'learning_rate': 1.9369806858494287e-05, 'epoch': 1.58}
{'loss': 0.0985, 'learning_rate': 1.92121403232164e-05, 'epoch': 1.97}
{'loss': 0.0746, 'learning_rate': 1.9054473787938513e-05, 'epoch': 2.36}
{'loss': 0.077, 'learning_rate': 1.8897437918801735e-05, 'epoch': 2.76}
{'loss': 0.0747, 'learning_rate': 1.8739771383523847e-05, 'epoch': 3.15}
{'loss': 0.0607, 'learning_rate': 1.858210484824596e-05, 'epoch': 3.55}
{'loss': 0.0648, 'learning_rate': 1.8424438312968072e-05, 'epoch': 3.94}


  0%|          | 0/1102 [00:00<?, ?it/s]

{'eval_loss': 0.047781217843294144, 'eval_accuracy': 0.971966859607309, 'eval_precision': 0.9169491525423729, 'eval_recall': 0.8789601949634444, 'eval_f1': 0.8975528826213189, 'eval_runtime': 30.9282, 'eval_samples_per_second': 284.886, 'eval_steps_per_second': 35.631, 'epoch': 3.94}
{'loss': 0.0604, 'learning_rate': 1.8266771777690185e-05, 'epoch': 4.34}
{'loss': 0.0587, 'learning_rate': 1.8109105242412298e-05, 'epoch': 4.73}
{'loss': 0.0635, 'learning_rate': 1.795159637366969e-05, 'epoch': 5.12}
{'loss': 0.0558, 'learning_rate': 1.779408750492708e-05, 'epoch': 5.52}
{'loss': 0.0525, 'learning_rate': 1.7636420969649194e-05, 'epoch': 5.91}
{'loss': 0.05, 'learning_rate': 1.7478754434371306e-05, 'epoch': 6.31}
{'loss': 0.0513, 'learning_rate': 1.7321245565628696e-05, 'epoch': 6.7}
{'loss': 0.0423, 'learning_rate': 1.716357903035081e-05, 'epoch': 7.09}
{'loss': 0.0417, 'learning_rate': 1.70060701616082e-05, 'epoch': 7.49}
{'loss': 0.0432, 'learning_rate': 1.684840362633031e-05, 'epoch': 

  0%|          | 0/1102 [00:00<?, ?it/s]

...

  0%|          | 0/1102 [00:00<?, ?it/s]

{'eval_loss': 0.010858706198632717, 'eval_accuracy': 0.9962546816479401, 'eval_precision': 0.9885807504078303, 'eval_recall': 0.9845653939886271, 'eval_f1': 0.9865689865689866, 'eval_runtime': 29.5587, 'eval_samples_per_second': 298.085, 'eval_steps_per_second': 37.282, 'epoch': 43.36}
{'loss': 0.0104, 'learning_rate': 2.5056365786361845e-06, 'epoch': 43.75}
{'loss': 0.013, 'learning_rate': 2.3481277098935753e-06, 'epoch': 44.15}
{'loss': 0.0095, 'learning_rate': 2.190461174615688e-06, 'epoch': 44.54}
{'loss': 0.0114, 'learning_rate': 2.0329523058730783e-06, 'epoch': 44.93}
{'loss': 0.0116, 'learning_rate': 1.8752857705951913e-06, 'epoch': 45.33}
{'loss': 0.0084, 'learning_rate': 1.7176192353173041e-06, 'epoch': 45.72}
{'loss': 0.0134, 'learning_rate': 1.559952700039417e-06, 'epoch': 46.12}
{'loss': 0.0116, 'learning_rate': 1.4024438312968075e-06, 'epoch': 46.51}
{'loss': 0.0116, 'learning_rate': 1.2449349625541981e-06, 'epoch': 46.91}
{'loss': 0.0087, 'learning_rate': 1.08726842727631

  0%|          | 0/1102 [00:00<?, ?it/s]

{'eval_loss': 0.010331220924854279, 'eval_accuracy': 0.9962546816479401, 'eval_precision': 0.987785016286645, 'eval_recall': 0.9853777416734363, 'eval_f1': 0.9865799105327369, 'eval_runtime': 29.6133, 'eval_samples_per_second': 297.536, 'eval_steps_per_second': 37.213, 'epoch': 47.3}
{'loss': 0.0127, 'learning_rate': 9.296018919984234e-07, 'epoch': 47.69}
{'loss': 0.0113, 'learning_rate': 7.719353567205361e-07, 'epoch': 48.09}
{'loss': 0.0103, 'learning_rate': 6.144264879779267e-07, 'epoch': 48.48}
{'loss': 0.0078, 'learning_rate': 4.5675995270003944e-07, 'epoch': 48.88}
{'loss': 0.0095, 'learning_rate': 2.992510839574301e-07, 'epoch': 49.27}
{'loss': 0.0097, 'learning_rate': 1.4158454867954278e-07, 'epoch': 49.66}
{'train_runtime': 17828.2412, 'train_samples_per_second': 56.904, 'train_steps_per_second': 7.115, 'train_loss': 0.03027781485782912, 'epoch': 50.0}


TrainOutput(global_step=126850, training_loss=0.03027781485782912, metrics={'train_runtime': 17828.2412, 'train_samples_per_second': 56.904, 'train_steps_per_second': 7.115, 'train_loss': 0.03027781485782912, 'epoch': 50.0})

In [13]:
samples = dataset_test.select_columns(['input_ids', 'attention_mask', 'labels'])
sample = samples[torch.randint(len(samples), (1,))]
with torch.no_grad():
    pred = model(**sample).logits.argmax(-1)
pred-sample['labels']

tensor([0], device='cuda:0')

In [14]:
res = []
for batch in tqdm(torch.utils.data.DataLoader(samples, batch_size=8)):
    pred = model(**batch).logits.argmax(-1)
    for p, l in zip(pred, batch['labels']):
        res.append((p.item(), l.item()))

  0%|          | 0/1102 [00:00<?, ?it/s]

In [15]:
pd.DataFrame(res, columns=['pred', 'label']).value_counts()

pred  label
0     0        7558
1     1        1205
0     1          26
1     0          22
Name: count, dtype: int64

```python
# TODO: check if this is correct
loss_weights = torch.tensor([neg_weight if x[0] == 1 else pos_weight for x in batch['label']])
loss_weights = torch.tensor([neg_weight, pos_weight]).to(device) # or this
model_v = "microsoft/MiniLM-L12-H384-uncased"

# Classical training loop:
import pickle
loss_log = []
epoch0 = 0
if os.path.exists("model/temp.state"):
    with open("model/temp.state", "rb") as f:
        state = pickle.load(f)
        loss_log = state['loss_log']
        epoch0 = state['epoch']

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
epochs = 10
batch_size = 4

model.train()
batches = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
for epoch in range(epoch0, epochs):
    for i, batch in enumerate(tqdm(batches)):
        class_weights = torch.tensor([neg_weight if x[0] == 1 else pos_weight for x in batch['labels']]).to(device)

        optimizer.zero_grad()
        loss = model.loss(batch, weights=class_weights)

        loss.backward()
        optimizer.step()
        loss_log.append(loss.cpu().detach().item())
        if (i+1) % (len(batches) // 20) == 0:
            print(np.mean(loss_log[-5000:]))
```